In [1]:
# imports:
import pyOSC3

import math
import time
from copy import copy
import numbers

from domblar.sc3.client import SC3Client

In [3]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider and run first cell

# 2. run this cell
client = SC3Client()


In [4]:
# defs


def build_mos_from_et(n, gen, edo, down=0, shift=0):
    notes = [0]
    cur = 0
    for i in range(n - down - 1):
        cur = (cur + gen) % edo
        notes.append(cur)
    cur = 0
    for i in range(down):
        cur = (cur - gen) % edo
        notes.append(cur)
    
    notes.sort()

    center = shift % len(notes)
    notes1 = [x - notes[center] for x in notes[center:]]
    notes2 = [x + edo - notes[center] for x in notes[:center]]
    return notes1 + notes2


def intervals_in_edo(sub_scale, scale=None):
    if scale is None:
        scale = list(range(edo))
    sub_scale_in_linear = []
    for note in sub_scale:
        in_oct = scale[note % len(scale)]
        oct_idx = note // len(scale)
        note_in_linear = in_oct + oct_idx * edo
        sub_scale_in_linear.append(note_in_linear)
    if (sub_scale_in_linear[0] + edo) not in sub_scale_in_linear:
        sub_scale_in_linear.append(sub_scale_in_linear[0] + edo)
    res = []
    for idx in range(len(sub_scale_in_linear) - 1):
        res.append(sub_scale_in_linear[idx + 1] - sub_scale_in_linear[idx])
    return res


def transpose(n, m):
    if isinstance(n, numbers.Number):
        return n + m
    if isinstance(n, list):
        return [transpose(note, m) for note in n]
    return n

def note_in_scale(sub_scale_orig, note):
    sub_scale = copy(sub_scale_orig)
    if edo in sub_scale:
        sub_scale = sub_scale[:-1]
    in_oct = sub_scale[note % len(sub_scale)]
    oct_idx = note // len(sub_scale)
    return (in_oct + oct_idx * edo)

def play(chords, dur=0.25, sus=None, delay=None, synth_idx=0):
    for chord_idx, chord in enumerate(chords):
        if type(chord) is not list:
            chord = [chord]
        for note_idx, note in enumerate(chord):
            freq = get_freq(note, scale, edo)
            send_note_dur = dur
            if sus:
                send_note_dur = sus
            timetag = time.time()
#             client.send_note(note_idx + (chord_idx % 2), freq=freq, dur=send_note_dur, timetag=timetag)
            client.send_note(synth_idx, freq=freq, dur=send_note_dur, timetag=timetag, channel=note_idx)
            if delay:
                time.sleep(delay)
        time.sleep(dur)

def play_voice(notes, timbre, dur=0.25, sus=None, delay=None):
    if type(notes) is not list:
        notes = [notes]
    for note in notes:
        freq = get_freq(note, scale, edo)
        send_note_dur = dur
        if sus:
            send_note_dur = sus
        timetag = time.time()
        client.send_note(timbre, freq=freq, dur=send_note_dur, timetag=timetag)
        time.sleep(dur)

def play_voices(voices, timbres, dur=0.25, sus=None, delay=None):
    # check preconditions
    for v in voices:
        assert (len(v) == len(voices[0]))
    assert (len(voices) == len(timbres))

    for note_idx in range(len(voices[0])):
        for v_idx, v in enumerate(voices):
            notes = v[note_idx]
            if not isinstance(notes, list):
                notes = [notes]
            for chord_note_idx, note in enumerate(notes):
                freq = get_freq(note, scale, edo)
                send_note_dur = dur
                if sus:
                    send_note_dur = sus
                timetag = time.time()
                client.send_note(timbres[v_idx], freq=freq, dur=send_note_dur*2, timetag=timetag, channel=chord_note_idx)
        time.sleep(dur)

def triad(sub_scale_orig, degree, quality='tertiary'):
    sub_scale = copy(sub_scale_orig)
    if edo in sub_scale:
        sub_scale = sub_scale[:-1]
    assert (len(sub_scale) == 7)
    if quality == 'tertiary':
        notes = [degree, degree + 2, degree + 4]
    elif quality == 'quartal':
        notes = [degree, degree + 3, degree + 6]
    chord = []
    for note in notes:
        in_oct = sub_scale[note % len(sub_scale)]
        oct_idx = note // len(sub_scale)
        chord.append(in_oct + oct_idx * edo)
    return chord

def play_triads(sub_scale, degrees, dur):
    chords = []
    for deg in degrees:
        chords.append(triad(sub_scale, deg))
    play(chords, dur=dur)

# parameters for Dexed
instruments = {}
instruments['brass/toto_africa'] = [ 1.0, 0.0, 1.0, 0.5, 0.54838711023331, 0.85714286565781, 1.0, 0.35353535413742, 0.0, 0.050505049526691, 0.0, 0.0, 0.0, 0.25, 0.4285714328289, 0.94949495792389, 0.67676764726639, 0.95959597826004, 0.60606062412262, 0.50505048036575, 0.50505048036575, 0.50505048036575, 0.50505048036575, 0.55555558204651, 0.24242424964905, 0.19191919267178, 0.55555558204651, 1.0, 0.86868685483932, 0.86868685483932, 0.0, 1.0, 0.0, 0.032258063554764, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.28571429848671, 1.0, 0.3737373650074, 0.34343433380127, 0.15151515603065, 0.70707070827484, 0.85858583450317, 0.0, 0.0, 0.0, 0.70707070827484, 0.0, 0.032258063554764, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.14285714924335, 1.0, 0.46464645862579, 0.35353535413742, 0.22222222387791, 0.50505048036575, 1.0, 0.86868685483932, 0.86868685483932, 0.0, 0.77777779102325, 0.0, 0.032258063554764, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714924335, 0.0, 0.14285714924335, 1.0, 0.66666668653488, 0.92929291725159, 0.22222222387791, 0.50505048036575, 0.53535354137421, 0.61616164445877, 0.62626260519028, 0.0, 0.79797977209091, 0.0, 0.032258063554764, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.4848484992981, 0.55555558204651, 0.22222222387791, 0.50505048036575, 0.98989897966385, 0.61616164445877, 0.62626260519028, 0.0, 0.70707070827484, 0.0, 0.096774190664291, 0.060606062412262, 0.4285714328289, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.77777779102325, 0.56565654277802, 0.20202019810677, 0.70707070827484, 1.0, 0.0, 0.0, 0.0, 0.79797977209091, 0.0, 0.22580644488335, 0.21212121844292, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0 ]
instruments['tub_bells'] = [ 1.0, 0.0, 1.0, 0.5, 0.12903225421906, 1.0, 0.0, 0.35353535413742, 0.0, 0.0, 0.0, 0.0, 0.20000000298023, 0.5, 0.14285714924335, 0.67676764726639, 0.95959597826004, 0.95959597826004, 0.60606062412262, 0.50505048036575, 0.50505048036575, 0.50505048036575, 0.50505048036575, 0.95959597826004, 0.33333334326744, 0.71717172861099, 0.25252524018288, 1.0, 0.0, 0.32323232293129, 0.0, 0.95959597826004, 0.0, 0.032258063554764, 0.0, 0.64285713434219, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.0, 1.0, 0.98989897966385, 0.12121212482452, 0.71717172861099, 0.28282827138901, 1.0, 0.0, 0.32323232293129, 0.0, 0.78787881135941, 0.0, 0.064516127109528, 0.75757575035095, 0.71428573131561, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.0, 1.0, 0.95959597826004, 0.33333334326744, 0.71717172861099, 0.25252524018288, 1.0, 0.0, 0.32323232293129, 0.0, 1.0, 0.0, 0.032258063554764, 0.0, 0.14285714924335, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.0, 1.0, 0.98989897966385, 0.12121212482452, 0.71717172861099, 0.28282827138901, 1.0, 0.0, 0.32323232293129, 0.0, 0.75757575035095, 0.0, 0.064516127109528, 0.75757575035095, 0.35714286565781, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.0, 1.0, 0.7676767706871, 0.78787881135941, 0.71717172861099, 0.70707070827484, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.064516127109528, 0.5151515007019, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.71428573131561, 1.0, 0.98989897966385, 0.91919189691544, 0.0, 0.28282827138901, 1.0, 0.0, 0.0, 0.0, 0.85858583450317, 0.0, 0.064516127109528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28571429848671, 0.0, 0.0, 1.0 ]
# this one has very weird release sound
instruments['harpsich1'] = [ 1.0, 0.0, 1.0, 0.5, 0.12903225421906, 0.14285714924335, 1.0, 0.35353535413742, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.28571429848671, 0.0, 0.0, 0.0, 0.0, 0.50505048036575, 0.50505048036575, 0.50505048036575, 0.50505048036575, 0.95959597826004, 0.28282827138901, 0.27272728085518, 0.47474747896194, 1.0, 0.90909093618393, 0.0, 0.0, 0.89898991584778, 0.0, 0.12903225421906, 0.0, 0.5, 0.49494948983192, 0.0, 0.0, 0.0, 0.0, 0.4285714328289, 0.0, 0.28571429848671, 1.0, 0.95959597826004, 0.72727274894714, 0.71717172861099, 1.0, 1.0, 0.9797979593277, 0.91919189691544, 0.98989897966385, 1.0, 0.0, 0.0, 0.0, 0.5, 0.49494948983192, 0.0, 0.0, 0.0, 0.0, 0.14285714924335, 0.0, 0.0, 1.0, 0.95959597826004, 0.28282827138901, 0.27272728085518, 0.47474747896194, 1.0, 0.90909093618393, 0.0, 0.0, 0.85858583450317, 0.0, 0.032258063554764, 0.0, 0.4285714328289, 0.49494948983192, 0.0, 0.0, 0.0, 0.0, 0.4285714328289, 0.0, 0.28571429848671, 1.0, 0.95959597826004, 0.72727274894714, 0.71717172861099, 1.0, 1.0, 0.9797979593277, 0.91919189691544, 0.98989897966385, 1.0, 0.0, 0.096774190664291, 0.0, 0.5, 0.64646464586258, 0.0, 0.46464645862579, 0.0, 0.0, 0.14285714924335, 0.0, 0.0, 1.0, 0.95959597826004, 0.28282827138901, 0.27272728085518, 0.47474747896194, 1.0, 0.90909093618393, 0.0, 0.0, 0.83838385343552, 0.0, 0.12903225421906, 0.0, 0.4285714328289, 0.49494948983192, 0.0, 0.0, 0.0, 0.0, 0.4285714328289, 0.0, 0.4285714328289, 1.0, 0.95959597826004, 0.72727274894714, 0.71717172861099, 1.0, 1.0, 0.9797979593277, 0.91919189691544, 0.98989897966385, 0.87878787517548, 0.0, 0.19354838132858, 0.0, 0.5, 0.64646464586258, 0.0, 0.55555558204651, 0.0, 0.0, 0.14285714924335, 0.0, 0.0, 1.0 ]
# todo: percussion
# kick, hihat, snare

def set_instrument(synth_idx, instrument):
    data = [synth_idx, 0] + instruments[instrument]
    client.send_msg('/set_instrument', data)
    return

def set_instruments():
#     for i in range(3):
    set_instrument(0, 'brass/toto_africa')


In [5]:
# play:

edo = 37
# # fixme: remove this scale, looks redundant
scale = list(range(edo))

set_instruments()

scales = {}
scales['orgone7'] = build_mos_from_et(7, gen=10, edo=edo, down=0, shift=0) + [edo]

cur_scale = scales['orgone7']
print(cur_scale)
print('steps -', intervals_in_edo(cur_scale))
while True:
    play(cur_scale, synth_idx=0, dur=0.3)


[0, 3, 10, 13, 20, 23, 30, 37]
steps - [3, 7, 3, 7, 3, 7, 7]


KeyboardInterrupt: 

In [ ]:
# cleanup
